In [19]:
import numpy as np
import pandas as pd
import math
import os
import requests
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.


True

In [20]:
df = pd.read_csv("./api_defs/raw_NetworkSupplyPointsCords.csv")

In [21]:
import googlemaps

GEO_GOOGLE_API_KEY = os.environ['GEO_GOOGLE_API_KEY']
gmaps = googlemaps.Client(key=GEO_GOOGLE_API_KEY)

def geocode(address: str):
    geocode_result = gmaps.geocode(address, region="nz")
    if len(geocode_result) == 0:
        return
    latlong = geocode_result[0]["geometry"]["location"]
    return latlong["lat"], latlong["lng"]

In [22]:
# https://gis.stackexchange.com/a/305013
def get_lat_lon(row):
    nztm_e = row["NZTM easting"]
    nztm_n = row["NZTM northing"]

    if (pd.isna(nztm_e) or pd.isna(nztm_n)):
        geocode_result = geocode(row["Description"])
        if geocode_result:
            lat, lon = geocode_result
            row["latitude"] = lat
            row["longitude"] = lon
        return row

    # Common variables for NZTM2000
    a = 6378137
    f = 1 / 298.257222101
    phizero = 0
    lambdazero = 173
    Nzero = 10000000
    Ezero = 1600000
    kzero = 0.9996

    # input Northing(Y); Easting(X) variables
    N = int(nztm_n)
    E = int(nztm_e)

    # Calculation: From NZTM to lat/Long
    b = a * (1 - f)
    esq = 2 * f - f ** 2
    Z0 = 1 - esq / 4 - 3 * (esq ** 2) / 64 - 5 * (esq ** 3) / 256
    A2 = 0.375 * (esq + esq ** 2 / 4 + 15 * (esq ** 3) / 128)
    A4 = 15 * ((esq ** 2) + 3 * (esq ** 3) / 4) / 256
    A6 = 35 * (esq ** 3) / 3072

    Nprime = N - Nzero
    mprime = Nprime / kzero
    smn = (a - b) / (a + b)
    G = a * (1 - smn) * (1 - (smn ** 2)) * (1 + 9 * (smn ** 2) /
                                            4 + 225 * (smn ** 4) / 64) * math.pi / 180.0
    sigma = mprime * math.pi / (180 * G)
    phiprime = sigma + (3 * smn / 2 - 27 * (smn ** 3) / 32) * math.sin(2 * sigma) + (21 * (smn ** 2) / 16 - 55 * (smn ** 4) / 32) * \
        math.sin(4 * sigma) + (151 * (smn ** 3) / 96) * math.sin(6 *
                                                                 sigma) + (1097 * (smn ** 4) / 512) * math.sin(8 * sigma)
    rhoprime = a * (1 - esq) / ((1 - esq * ((math.sin(phiprime)) ** 2)) ** 1.5)
    upsilonprime = a / math.sqrt(1 - esq * ((math.sin(phiprime)) ** 2))

    psiprime = upsilonprime / rhoprime
    tprime = math.tan(phiprime)
    Eprime = E - Ezero
    chi = Eprime / (kzero * upsilonprime)
    term_1 = tprime * Eprime * chi / (kzero * rhoprime * 2)
    term_2 = term_1 * (chi ** 2) / 12 * (-4 * (psiprime ** 2) +
                                         9 * psiprime * (1 - (tprime ** 2)) + 12 * (tprime ** 2))
    term_3 = tprime * Eprime * (chi ** 5) / (kzero * rhoprime * 720) * (8 * (psiprime ** 4) * (11 - 24 * (tprime ** 2)) - 12 * (psiprime ** 3) * (21 - 71 * (
        tprime ** 2)) + 15 * (psiprime ** 2) * (15 - 98 * (tprime ** 2) + 15 * (tprime ** 4)) + 180 * psiprime * (5 * (tprime ** 2) - 3 * (tprime ** 4)) + 360 * (tprime ** 4))
    term_4 = tprime * Eprime * (chi ** 7) / (kzero * rhoprime * 40320) * (
        1385 + 3633 * (tprime ** 2) + 4095 * (tprime ** 4) + 1575 * (tprime ** 6))
    term1 = chi * (1 / math.cos(phiprime))
    term2 = (chi ** 3) * (1 / math.cos(phiprime)) / \
        6 * (psiprime + 2 * (tprime ** 2))
    term3 = (chi ** 5) * (1 / math.cos(phiprime)) / 120 * (-4 * (psiprime ** 3) * (1 - 6 * (tprime ** 2)) +
                                                           (psiprime ** 2) * (9 - 68 * (tprime ** 2)) + 72 * psiprime * (tprime ** 2) + 24 * (tprime ** 4))
    term4 = (chi ** 7) * (1 / math.cos(phiprime)) / 5040 * (61 + 662 *
                                                            (tprime ** 2) + 1320 * (tprime ** 4) + 720 * (tprime ** 6))

    row["latitude"] = (phiprime - term_1 + term_2 -
                       term_3 + term_4) * 180 / math.pi
    row["longitude"] = lambdazero + 180 / \
        math.pi * (term1 - term2 + term3 - term4)
    return row


In [23]:
df = df.apply(get_lat_lon, axis=1)

In [24]:
df = df[df['latitude'].notna()]

In [25]:
df = df.drop(labels=["NZTM easting","NZTM northing"], axis=1)
df = df.rename(columns={
    "POC code": "connection_code",
    "Description": "address",
    "Network reporting region ID": "network_region_id",
    "Network reporting region": "network_region_name",
    "Zone": "network_region_zone",
    "latitude": "latitude",
    "longitude": "longitude"
})
df

,address,network_region_name,network_region_id,connection_code,network_region_zone,latitude,longitude
0,ALBURY,South Canterbury (Alpine Energy),32.0,ABY0111,LSI,-44.251511,170.800381
1,57 Hall Road Kerikeri,Bay of Islands (Top Energy),1.0,AKK0011,UNI,-35.236532,173.950672
2,AUCKLAND AIRPORT,Auckland (Vector),4.0,AKL0331,UNI,-36.962752,174.827615
3,ALBANY,Waitemata (Vector),3.0,ALB0331,UNI,-36.739090,174.690134
4,ALBANY,Waitemata (Vector),3.0,ALB1101,UNI,-36.739090,174.690134
...,...,...,...,...,...,...,...
2126,WESTFIELD WESTCITY,Waitemata (Vector),3.0,WWC0011,UNI,-36.886090,174.660301
2127,Westwind,Wellington (Wellington Electricity),23.0,WWD1102,LNI,-41.295618,174.660496
2128,Westwind,Wellington (Wellington Electricity),23.0,WWD1103,LNI,-41.295618,174.660496
2129,WHAREWAKA,Taupo (Unison Networks),13.0,WWK0111,CNI,-38.625688,176.105440


In [26]:
# Remove duplicates
df = df.drop_duplicates(subset=['connection_code'])

In [27]:
df[["connection_code","address","network_region_id","network_region_name","network_region_zone","latitude","longitude"]].to_csv("./api_defs/clean_NetworkSupplyPointsCords.csv")